# IMDB & Wikimedia — Final Project (RAM-safe)

This notebook answers the **IMDB analysis questions (Q1 → Q14)** for the Final Project, using the public datasets from:  
https://datasets.imdbws.com/

Because IMDB files are large, the notebook is designed to be **RAM-safe**:
- reads files **in chunks** (`chunksize`)
- loads only required columns (`usecols`)
- filters rows **as early as possible** (especially for `title.akas`)

> IMDB encodes missing values as `\N`. We treat those as `NaN`.


## Questions (Q1 → Q14)

1. Download IMDB datasets from the official source using a notebook cell (manual upload allowed only if documented).  
2. How many total people are in the dataset?  
3. What is the earliest year of birth?  
4. How many years ago was this person born?  
5. Using only IMDB data, determine whether this birth year is plausible.  
6. Explain the reasoning (comment or markdown).  
7. What is the most recent year of birth?  
8. What percentage of people have no listed birth year?  
9. What is the runtime of the longest **short** after 1900?  
10. What is the runtime of the shortest **movie** after 1900?  
11. List all genres represented.  
12. What is the highest-rated **comedy movie** (tie-breaker: most votes)?  
13. Who was the director of that movie?  
14. List the alternate titles for that movie.  


## 0) Setup

In [3]:
import gc
from pathlib import Path
from datetime import datetime

import pandas as pd
import numpy as np
from IPython.display import display, Markdown

pd.set_option("display.max_columns", 80)

DATA_DIR = Path("data")
RAW_DIR = DATA_DIR / "raw"
RAW_DIR.mkdir(parents=True, exist_ok=True)

CURRENT_YEAR = datetime.now().year

def answer_md(text: str):
    """Render a clean 'final answer' block as Markdown."""
    display(Markdown(text))

print("Raw data folder:", RAW_DIR.resolve())
print("Current year:", CURRENT_YEAR)


Raw data folder: /content/data/raw
Current year: 2025


## 1) Download IMDB datasets (Question 1)

This cell downloads the required IMDB `.tsv.gz` files automatically.

If a download fails (network policy, proxy, etc.), you may manually place files into:
- `data/raw/`

If you do so, keep the download error output as evidence and mention it in your README/notebook.


In [4]:
import urllib.request
from pathlib import Path # Added this import

IMDB_BASE = "https://datasets.imdbws.com/"
REQUIRED_FILES = [
    "name.basics.tsv.gz",
    "title.basics.tsv.gz",
    "title.ratings.tsv.gz",
    "title.crew.tsv.gz",
    "title.akas.tsv.gz",  # very large: ALWAYS filter by tconst when reading
]

def download_file(url: str, out_path: Path, overwrite: bool = False) -> None:
    out_path.parent.mkdir(parents=True, exist_ok=True)
    if out_path.exists() and not overwrite:
        return
    print("Downloading:", url)
    urllib.request.urlretrieve(url, out_path)

for fname in REQUIRED_FILES:
    fpath = RAW_DIR / fname
    if fpath.exists():
        print("OK:", fname)
        continue
    try:
        download_file(IMDB_BASE + fname, fpath)
        print("DONE:", fname)
    except Exception as e:
        print(f"⚠️ Could not download {fname}: {e}")
        print(f"   → Manual option: place the file at {fpath}")

Downloading: https://datasets.imdbws.com/name.basics.tsv.gz
DONE: name.basics.tsv.gz
Downloading: https://datasets.imdbws.com/title.basics.tsv.gz
DONE: title.basics.tsv.gz
Downloading: https://datasets.imdbws.com/title.ratings.tsv.gz
DONE: title.ratings.tsv.gz
Downloading: https://datasets.imdbws.com/title.crew.tsv.gz
DONE: title.crew.tsv.gz
Downloading: https://datasets.imdbws.com/title.akas.tsv.gz
DONE: title.akas.tsv.gz


## 2) RAM-safe helper functions

We never read the full largest files into RAM. Instead, we iterate over chunks and keep only what we need.


In [5]:
NA_VALUES = ["\\N"]

def read_tsv_chunks(path: Path, usecols=None, chunksize: int = 200_000, dtype=None):
    """Yield DataFrame chunks from a .tsv.gz file."""
    return pd.read_csv(
        path,
        sep="\t",
        compression="gzip",
        na_values=NA_VALUES,
        keep_default_na=True,
        usecols=usecols,
        chunksize=chunksize,
        low_memory=False,
        dtype=dtype,
    )

def load_filtered_tsv(path: Path, usecols, row_filter=None, chunksize: int = 200_000, dtype=None):
    """Load only selected rows (after optional filtering) chunk-by-chunk."""
    out = []
    for chunk in read_tsv_chunks(path, usecols=usecols, chunksize=chunksize, dtype=dtype):
        if row_filter is not None:
            chunk = row_filter(chunk)
        if len(chunk):
            out.append(chunk)
    if not out:
        return pd.DataFrame(columns=usecols)
    return pd.concat(out, ignore_index=True)

def to_int(series):
    return pd.to_numeric(series, errors="coerce").astype("Int64")

def to_float(series):
    return pd.to_numeric(series, errors="coerce").astype("Float64")


---
# Part A — IMDB Questions (Q2 → Q14)


## Q2) How many total people are in the dataset?
Source: `name.basics.tsv.gz`

In [6]:
people_path = RAW_DIR / "name.basics.tsv.gz"
assert people_path.exists(), f"Missing file: {people_path}"

total_people = 0
for chunk in read_tsv_chunks(people_path, usecols=["nconst"], chunksize=500_000):
    total_people += len(chunk)

total_people


14964905

In [7]:
answer_md(f"**Answer (Q2):** The dataset contains **{total_people:,} people**.")

**Answer (Q2):** The dataset contains **14,964,905 people**.

## Q3–Q6) Earliest birth year, years ago, and plausibility (IMDB-only)

We compute the minimum `birthYear` and then check **internal consistency** using only IMDB fields:
- If `deathYear < birthYear` → inconsistent
- If any `knownForTitles` has `startYear < birthYear` → suspicious

This does not prove real-world accuracy, but it is a valid IMDB-only consistency check (as requested).


In [8]:
people_cols = ["nconst", "primaryName", "birthYear", "deathYear", "knownForTitles"]
df_people = load_filtered_tsv(people_path, usecols=people_cols, chunksize=300_000)

df_people["birthYear"] = to_int(df_people["birthYear"])
df_people["deathYear"] = to_int(df_people["deathYear"])

earliest_birth_year = int(df_people["birthYear"].min())
earliest_people = df_people[df_people["birthYear"] == earliest_birth_year].copy()

earliest_birth_year, earliest_people.head(3)


(4,
            nconst         primaryName  birthYear  deathYear  \
 737939  nm0784172  Lucio Anneo Seneca          4         65   
 
                                  knownForTitles  
 737939  tt0043802,tt0218822,tt0049203,tt0972562  )

In [9]:
from datetime import datetime
CURRENT_YEAR = datetime.now().year
years_ago = CURRENT_YEAR - earliest_birth_year
years_ago


2021

In [11]:
# Plausibility check for the first matching person (IMDB-only)
p = earliest_people.iloc[0]
birth = int(p["birthYear"]) if pd.notna(p["birthYear"]) else None
death = int(p["deathYear"]) if pd.notna(p["deathYear"]) else None

issues = []

if birth is not None and death is not None and death < birth:
    issues.append("deathYear < birthYear")

titles_path = RAW_DIR / "title.basics.tsv.gz"
assert titles_path.exists(), f"Missing file: {titles_path}"

known = []
if pd.notna(p.get("knownForTitles")):
    known = [x.strip() for x in str(p["knownForTitles"]).split(",") if x.strip()]

df_known_titles = load_filtered_tsv(
    titles_path,
    usecols=["tconst", "primaryTitle", "startYear", "titleType"],
    row_filter=lambda df: df[df["tconst"].isin(known)], # Fixed: Filter the DataFrame directly
    chunksize=300_000,
)
df_known_titles["startYear"] = to_int(df_known_titles["startYear"])

if birth is not None and len(df_known_titles):
    min_title_year = df_known_titles["startYear"].min()
    if pd.notna(min_title_year) and int(min_title_year) < birth:
        issues.append(f"knownForTitles contains startYear {int(min_title_year)} < birthYear {birth}")

summary_q5 = {
    "nconst": p["nconst"],
    "primaryName": p["primaryName"],
    "birthYear": birth,
    "deathYear": death,
    "knownForTitles_count": len(known),
    "issues_found": issues,
    "birthYear_plausible": (len(issues) == 0),
}
summary_q5

{'nconst': 'nm0784172',
 'primaryName': 'Lucio Anneo Seneca',
 'birthYear': 4,
 'deathYear': 65,
 'knownForTitles_count': 4,
 'issues_found': [],
 'birthYear_plausible': True}

In [12]:
answer_md(
f"**Answer (Q3):** The earliest birth year is **{earliest_birth_year}**.\n\n"
f"**Answer (Q4):** This is about **{years_ago} years ago** (based on the current year {CURRENT_YEAR}).\n\n"
f"**Answer (Q5):** Plausible (IMDB-only) = **{summary_q5['birthYear_plausible']}**.\n\n"
f"**Answer (Q6):** We checked `deathYear >= birthYear` and verified that `knownForTitles` does not include titles released before the birth year. "
f"Issues found: **{summary_q5['issues_found']}**."
)

**Answer (Q3):** The earliest birth year is **4**.

**Answer (Q4):** This is about **2021 years ago** (based on the current year 2025).

**Answer (Q5):** Plausible (IMDB-only) = **True**.

**Answer (Q6):** We checked `deathYear >= birthYear` and verified that `knownForTitles` does not include titles released before the birth year. Issues found: **[]**.

## Q7–Q8) Most recent birth year & percentage missing birthYear

In [13]:
most_recent_birth_year = int(df_people["birthYear"].max())
pct_missing_birth = float(df_people["birthYear"].isna().mean() * 100)

most_recent_birth_year, pct_missing_birth


(2025, 95.57803407372116)

In [14]:
answer_md(
f"**Answer (Q7):** The most recent birth year is **{most_recent_birth_year}**.\n\n"
f"**Answer (Q8):** **{pct_missing_birth:.2f}%** of people have no listed birth year."
)

**Answer (Q7):** The most recent birth year is **2025**.

**Answer (Q8):** **95.58%** of people have no listed birth year.

## Q9–Q10) Runtime extremes after 1900

We scan `title.basics` in chunks and keep only titles with `startYear > 1900`.

- Q9: longest `short`
- Q10: shortest `movie` (runtime > 0)


In [15]:
usecols_titles = ["tconst", "titleType", "primaryTitle", "startYear", "runtimeMinutes", "genres"]

longest_short = None
longest_short_row = None

shortest_movie = None
shortest_movie_row = None

for chunk in read_tsv_chunks(titles_path, usecols=usecols_titles, chunksize=300_000):
    chunk["startYear"] = to_int(chunk["startYear"])
    chunk["runtimeMinutes"] = pd.to_numeric(chunk["runtimeMinutes"], errors="coerce")

    chunk = chunk[chunk["startYear"].notna() & (chunk["startYear"] > 1900)]

    shorts = chunk[(chunk["titleType"] == "short") & chunk["runtimeMinutes"].notna()]
    if len(shorts):
        idx = shorts["runtimeMinutes"].idxmax()
        val = float(shorts.loc[idx, "runtimeMinutes"])
        if (longest_short is None) or (val > longest_short):
            longest_short = val
            longest_short_row = shorts.loc[idx, ["tconst","primaryTitle","startYear","runtimeMinutes","genres"]].to_dict()

    movies = chunk[(chunk["titleType"] == "movie") & chunk["runtimeMinutes"].notna() & (chunk["runtimeMinutes"] > 0)]
    if len(movies):
        idx = movies["runtimeMinutes"].idxmin()
        val = float(movies.loc[idx, "runtimeMinutes"])
        if (shortest_movie is None) or (val < shortest_movie):
            shortest_movie = val
            shortest_movie_row = movies.loc[idx, ["tconst","primaryTitle","startYear","runtimeMinutes","genres"]].to_dict()

(longest_short, longest_short_row, shortest_movie, shortest_movie_row)


(1311.0,
 {'tconst': 'tt35509411',
  'primaryTitle': 'Our First Day',
  'startYear': 2025,
  'runtimeMinutes': 1311.0,
  'genres': 'Drama,Short'},
 1.0,
 {'tconst': 'tt0025166',
  'primaryTitle': "George White's Scandals",
  'startYear': 1934,
  'runtimeMinutes': 1.0,
  'genres': 'Comedy,Musical,Romance'})

In [16]:
answer_md(
f"**Answer (Q9):** Longest short runtime after 1900 = **{longest_short} minutes**.\n\n"
f"Details: `{longest_short_row}`\n\n"
f"**Answer (Q10):** Shortest movie runtime after 1900 = **{shortest_movie} minutes**.\n\n"
f"Details: `{shortest_movie_row}`"
)

**Answer (Q9):** Longest short runtime after 1900 = **1311.0 minutes**.

Details: `{'tconst': 'tt35509411', 'primaryTitle': 'Our First Day', 'startYear': 2025, 'runtimeMinutes': 1311.0, 'genres': 'Drama,Short'}`

**Answer (Q10):** Shortest movie runtime after 1900 = **1.0 minutes**.

Details: `{'tconst': 'tt0025166', 'primaryTitle': "George White's Scandals", 'startYear': 1934, 'runtimeMinutes': 1.0, 'genres': 'Comedy,Musical,Romance'}`

## Q11) List all genres represented

In [17]:
genres_set = set()

for chunk in read_tsv_chunks(titles_path, usecols=["genres"], chunksize=400_000):
    for s in chunk["genres"].dropna().astype(str):
        for item in s.split(","):
            item = item.strip()
            if item:
                genres_set.add(item)

genres_list = sorted(genres_set)
len(genres_list), genres_list[:40]


(28,
 ['Action',
  'Adult',
  'Adventure',
  'Animation',
  'Biography',
  'Comedy',
  'Crime',
  'Documentary',
  'Drama',
  'Family',
  'Fantasy',
  'Film-Noir',
  'Game-Show',
  'History',
  'Horror',
  'Music',
  'Musical',
  'Mystery',
  'News',
  'Reality-TV',
  'Romance',
  'Sci-Fi',
  'Short',
  'Sport',
  'Talk-Show',
  'Thriller',
  'War',
  'Western'])

In [18]:
answer_md(
f"**Answer (Q11):** There are **{len(genres_list)} distinct genres**.\n\n"
f"Example (first 40): `{genres_list[:40]}`"
)

**Answer (Q11):** There are **28 distinct genres**.

Example (first 40): `['Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']`

## Q12) Highest-rated comedy **movie**

Rules:
- select titles where `titleType == "movie"` and genre contains **Comedy**
- maximize `averageRating`
- tie-breaker: maximize `numVotes`


In [19]:
ratings_path = RAW_DIR / "title.ratings.tsv.gz"
assert ratings_path.exists(), f"Missing file: {ratings_path}"

# 1) Collect comedy movies (RAM-safe)
comedy_rows = []
for chunk in read_tsv_chunks(titles_path, usecols=["tconst","titleType","primaryTitle","startYear","genres"], chunksize=300_000):
    chunk = chunk[chunk["titleType"] == "movie"].copy()
    chunk["genres"] = chunk["genres"].astype(str)
    chunk = chunk[chunk["genres"].str.contains(r"(^|,)Comedy(,|$)", regex=True, na=False)]
    if len(chunk):
        comedy_rows.append(chunk[["tconst","primaryTitle","startYear","genres"]])

df_comedy = pd.concat(comedy_rows, ignore_index=True) if comedy_rows else pd.DataFrame(columns=["tconst","primaryTitle","startYear","genres"])
df_comedy.shape


/tmp/ipython-input-1861117221.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  chunk = chunk[chunk["genres"].str.contains(r"(^|,)Comedy(,|$)", regex=True, na=False)]
/tmp/ipython-input-1861117221.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  chunk = chunk[chunk["genres"].str.contains(r"(^|,)Comedy(,|$)", regex=True, na=False)]
/tmp/ipython-input-1861117221.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  chunk = chunk[chunk["genres"].str.contains(r"(^|,)Comedy(,|$)", regex=True, na=False)]
/tmp/ipython-input-1861117221.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  chunk = chunk[chunk["genres"].str.contains(r"(^|,)Comed

(122520, 4)

In [20]:
# 2) Load ratings
df_ratings = pd.concat(
    read_tsv_chunks(ratings_path, usecols=["tconst","averageRating","numVotes"], chunksize=500_000),
    ignore_index=True
)
df_ratings["averageRating"] = pd.to_numeric(df_ratings["averageRating"], errors="coerce")
df_ratings["numVotes"] = pd.to_numeric(df_ratings["numVotes"], errors="coerce").astype("Int64")

# 3) Join + pick best
df_best = df_comedy.merge(df_ratings, on="tconst", how="inner").dropna(subset=["averageRating","numVotes"])
best_row = df_best.sort_values(["averageRating","numVotes"], ascending=[False, False]).head(1)

best_row


,tconst,primaryTitle,startYear,genres,averageRating,numVotes
79200,tt8458418,O La La,2018.0,Comedy,10.0,6


In [21]:
best_tconst = best_row.iloc[0]["tconst"]
best_title = best_row.iloc[0]["primaryTitle"]
best_rating = float(best_row.iloc[0]["averageRating"])
best_votes = int(best_row.iloc[0]["numVotes"])

answer_md(f"**Answer (Q12):** **{best_title}** (`{best_tconst}`) with rating **{best_rating}** and **{best_votes:,} votes**.")


**Answer (Q12):** **O La La** (`tt8458418`) with rating **10.0** and **6 votes**.

## Q13) Who was the director of the movie?

We use:
- `title.crew.tsv.gz` to get the director `nconst` list for `best_tconst`
- `name.basics.tsv.gz` to map `nconst` → `primaryName`


In [24]:
crew_path = RAW_DIR / "title.crew.tsv.gz"
assert crew_path.exists(), f"Missing file: {crew_path}"

df_crew = load_filtered_tsv(
    crew_path,
    usecols=["tconst","directors"],
    row_filter=lambda df: df[df["tconst"] == best_tconst],
    chunksize=500_000,
)
df_crew

,tconst,directors
0,tt8458418,nm7709412


In [25]:
directors_ids = []
if len(df_crew) and pd.notna(df_crew.loc[0, "directors"]):
    directors_ids = [x.strip() for x in str(df_crew.loc[0, "directors"]).split(",") if x.strip()]

directors_ids


['nm7709412']

In [29]:
if directors_ids:
    df_directors = load_filtered_tsv(
        people_path,
        usecols=["nconst","primaryName","birthYear","deathYear"],
        row_filter=lambda df: df[df["nconst"].isin(directors_ids)],
        chunksize=400_000,
    )
else:
    df_directors = pd.DataFrame(columns=["nconst","primaryName","birthYear","deathYear"])

df_directors

,nconst,primaryName,birthYear,deathYear
0,nm7709412,Sripad Pai,1965.0,NaN


In [30]:
director_names = df_directors["primaryName"].dropna().tolist() if len(df_directors) else []
answer_md(f"**Answer (Q13):** Director(s): **{', '.join(director_names) if director_names else 'Not available in IMDB crew data'}**.")

**Answer (Q13):** Director(s): **Sripad Pai**.

## Q14) List alternate titles for the movie

`title.akas.tsv.gz` is very large, so we strictly filter rows where `titleId == best_tconst`.


In [33]:
akas_path = RAW_DIR / "title.akas.tsv.gz"
assert akas_path.exists(), f"Missing file: {akas_path}"

akas_cols = ["titleId","ordering","title","region","language","types","attributes","isOriginalTitle"]

df_akas = load_filtered_tsv(
    akas_path,
    usecols=akas_cols,
    row_filter=lambda df: df[df["titleId"] == best_tconst],
    chunksize=500_000,
)

df_akas.sort_values("ordering").head(50)

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt8458418,1,O La La,NaN,NaN,original,NaN,1
1,tt8458418,2,O La La,IN,en,imdbDisplay,NaN,0


In [34]:
answer_md(
f"**Answer (Q14):** Found **{len(df_akas)} alternate title rows** for `{best_tconst}`.\n\n"
f"See the table above for region/language variants and original-title flags."
)

**Answer (Q14):** Found **2 alternate title rows** for `tt8458418`.

See the table above for region/language variants and original-title flags.